In [1]:
import json
import os
import pandas as pd
import time

from datetime import datetime
from tqdm.auto import tqdm
from web3 import Web3

# Set up your access to the blockchain with hardhat/alchemy/etc

In [2]:
PROVIDER = os.environ.get('ETH_PROVIDER_URL')
print(PROVIDER is not None) # Check that somethings populated, print this if you want to make sure...
w3 = Web3(Web3.HTTPProvider(PROVIDER))

True


In [3]:
w3.is_connected()

True

In [4]:
snapshot_block = 18780000 # Dec-13-2023 09:43:47 PM +UTC
block_details = w3.eth.get_block(snapshot_block)
display(block_details)

AttributeDict({'number': 18780000,
 'hash': HexBytes('0x1d4646fe6a909f657fc67d60a8d01d6cefd02fa9343aab02d2683b3df2ebcba0'),
 'transactions': [HexBytes('0x15e13956549dbdad45c8a5a96640ab840d9af4afdcbc00e9adf38340d38619ec'),
  HexBytes('0x61c26b4f9831e88811eafccab063f85812dc7d76042fb4c52c363adf5eb0c183'),
  HexBytes('0x01e0d62961c76c48e7c58316d9b3db01ab25c248028fb34fef42cc2af899fb63'),
  HexBytes('0x7aea881a3759c8c4fdc4bf7aa51c70dd716a39bbdf31d9068dce9d6af3f3d7bb'),
  HexBytes('0xf1cf4ab3c0212fc75b518ebb5ea2c60bd11a4550567e3323627fd87a16955855'),
  HexBytes('0x1a68da7a9cd2f836a4f20bbcd79e9125542bf4fd290255264b902985bfe057fd'),
  HexBytes('0xad74639533f50b0f21beec3fa3e1d50ac8ed462187955f0f2af8647256aba130'),
  HexBytes('0x27e09b98cf641412af3234d24ee393bde27ce3b9ed7cd2c98a92fb5d936aa8c2'),
  HexBytes('0x41bed468acf76c45c5e6f0b1c2e51a088ca622151fcc849419973f3888392218'),
  HexBytes('0x99c7ed4d4558f1adc74f1c4ded9e06a63494ace23eed541aec7ff87a1bb57c2a'),
  HexBytes('0x2731c5367ef0f4149603e54bcd

In [5]:
print('block',block_details['number'])
print('time',block_details['timestamp'])
print(datetime.utcfromtimestamp(block_details['timestamp']))

block 18780000
time 1702503827
2023-12-13 21:43:47


# Get owners of Acclimated, Wild and Jumpport cats

In [6]:
# Contracts 

# https://mooncat.community/build-with-mooncats
mooncat_traits_addr = '0x9330BbfBa0C8FdAf0D93717E4405a410a6103cC2'
mooncat_traits_abi = [{"inputs":[{"internalType":"uint256","name":"rescueOrder","type":"uint256"}],"name":"ownerOf","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"}]

jumpport_address = '0xF4d150F3D03Fa1912aad168050694f0fA0e44532'
jumpport_abi = [{"inputs":[{"internalType":"address","name":"tokenAddress","type":"address"},{"internalType":"uint256","name":"tokenId","type":"uint256"}],"name":"ownerOf","outputs":[{"internalType":"address","name":"owner","type":"address"}],"stateMutability":"view","type":"function"}]

acclimator_address = '0xc3f733ca98E0daD0386979Eb96fb1722A1A05E69'

In [7]:
mooncat_owners = {}

In [8]:
mooncat_traits_contract = w3.eth.contract(address=mooncat_traits_addr,abi=mooncat_traits_abi)

In [9]:
TOTAL_CATS = 25440

for cat_id in tqdm(range(TOTAL_CATS)):
    if cat_id in mooncat_owners: continue # for retries
    try:
        owner_address = mooncat_traits_contract.functions.ownerOf(cat_id).call(block_identifier=snapshot_block)
    except Exception as e:
        print(f'error with {cat_id}')
        continue
    mooncat_owners[cat_id] = owner_address
    with open(f'mooncat_id_owner_{snapshot_block}.csv','a') as f:
        f.write(f'{cat_id},{owner_address}\n')

  0%|          | 0/25440 [00:00<?, ?it/s]

error with 1074
error with 5998
error with 19978
error with 23304
error with 24222
error with 24904


In [10]:
# Errors
# 1074 - stuck in acclimator contract :(
# 5998 - stuck in acclimator contract :(
# 19978 - stuck in acclimator contract :(
# 23304 - stuck in acclimator contract :(
# 24222 - stuck in acclimator contract :(
# 24904 - stuck in acclimator contract :(

In [11]:
owner_table = pd.Series(mooncat_owners)
owner_table.value_counts()

0x7C40c393DC0f283F318791d746d894DdD3693572    1355
0xe307C2d3236bE4706E5D7601eE39F16d796d8195    1098
0xEF87243Fe29c2453F87549e02982108a3B832952     354
0xB11c61923861695eB47CFB3CCc473173c9cF7A28     326
0x1BA8D2BBCe4C2fd4885F032E298C80aB728504B7     280
                                              ... 
0xA3991cE46a75Fb4f108b7adC758499118159069b       1
0x5eEBDcDCfb30e8b7c490ae1A3c8ff0FA25BcD151       1
0xE6ac306031aC3d6C9d3c8a2bbE34FD2a4115f0C8       1
0x2e8E5CDfF08A22CCb037cd9ecB56Ad4c96F89e78       1
0x55e1490a1878D0B61811726e2cB96560022E764c       1
Name: count, Length: 6529, dtype: int64

In [12]:
jumpport_contract = w3.eth.contract(address=jumpport_address,abi=jumpport_abi)

In [13]:
jumpport_cat_owners = {}

In [14]:
print(f'Number in jumpport: {owner_table[owner_table==jumpport_address].shape[0]}')

Number in jumpport: 45


In [15]:
for cat_id in tqdm(owner_table[owner_table==jumpport_address].index):
    jumpport_owner = jumpport_contract.functions.ownerOf(acclimator_address,cat_id).call(block_identifier=snapshot_block)
    jumpport_cat_owners[cat_id] = jumpport_owner

  0%|          | 0/45 [00:00<?, ?it/s]

In [16]:
jumpport_table = pd.Series(jumpport_cat_owners)
jumpport_table.value_counts()

0xE59be815853613a929a7cD7F35215Df967055D69    10
0xa8E7A10D2eB9F59840a7eD6D4484956124588cF3     8
0xF0F225E01cf084d3C246Ad595Ab60d0D4F5b4c78     4
0xf69A1AA4ec5A5715988f35828B285A3A032ec6A8     3
0x713f90407fc49f0BC59A6661C23FB67692a80E6d     3
0x0C3B6Ff6B8543c8EA07423126df362E3E7727E44     2
0xa49dcfD59f39DfadA7EB11f60f173187B5401a06     2
0xE22c1d3B82B028eB0ddf481f64dF9A077c25278e     1
0xAd82E9e0D25bE8943BF4483C1D91e4e839d5533D     1
0xB47229E4cA179ad0211B3fE8036550d936A59401     1
0x9DDe8E217fa8Be81EeC7D9fA3A4bD80bf90DCfdF     1
0x8907FF531885985b2A015B442f139810D8715623     1
0x2edE52C0bb8FCAA98BFD2C8068F9D155dcce166d     1
0x767a3604750858fD489A50085D441d2D0D44984d     1
0x873dFcbBc61Ef7ebADb652adB98df9f69399B70b     1
0x35105eB9FBd3C3F03c77D8Bd4F4480FC0Cf6b02a     1
0xaF0E9bF133709dFdc53a7e95e8fA3Ebc6C5B4008     1
0xd233b4C22989CA802Fc79289c220B35d03E62dE7     1
0x1189f4016460543Ac15Cfb71b393c08682073a76     1
0x8B0cBF0Aa84B9Dc05B9385b5D249c487aC38EA03     1
Name: count, dtype: 

In [17]:
# Check before updating from jumport
combined_table = owner_table.copy()
combined_table[combined_table==jumpport_address]

1057     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
1675     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
3429     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
5856     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
6155     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
6538     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
7551     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
7594     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
7825     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
7842     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
8168     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
8978     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
9303     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
9551     0xF4d150F3D03Fa1912aad168050694f0fA0e44532
10213    0xF4d150F3D03Fa1912aad168050694f0fA0e44532
10683    0xF4d150F3D03Fa1912aad168050694f0fA0e44532
11005    0xF4d150F3D03Fa1912aad168050694f0fA0e44532
12164    0xF4d150F3D03Fa1912aad168050694f0fA0e44532
12349    0xF4d150F3D03Fa1912aad168050694f0fA0e44532
12708    0xF

In [18]:
combined_table.loc[jumpport_table.index] = jumpport_table
combined_table

0        0x5fCf881976eDaF9FAa5E9D7f2E08eE8Afd163372
1        0x2e961d40628a78Ab1F5Ce3c1a478B71787D7195f
2        0x2621ea417659Ad69bAE66af05ebE5788E533E5e7
3        0x2621ea417659Ad69bAE66af05ebE5788E533E5e7
4        0x2621ea417659Ad69bAE66af05ebE5788E533E5e7
                            ...                    
25435    0x7C40c393DC0f283F318791d746d894DdD3693572
25436    0x5C25BCBA2339F4b0b5f89a61ec87Ec9831c09e45
25437    0x5d0dfB525a392A904E9F1dA78DC1DD86253D2c9e
25438    0x0fc2A0caC9C3FC884809a55CCEAd18b70C0E37c9
25439    0x0793886aBCc2E243D890df1E76E1B84220f25615
Length: 25434, dtype: object

In [19]:
# Should be empty
combined_table[combined_table==jumpport_address]

Series([], dtype: object)

In [20]:
# Updated counts
combined_table.value_counts()

0x7C40c393DC0f283F318791d746d894DdD3693572    1355
0xe307C2d3236bE4706E5D7601eE39F16d796d8195    1098
0xEF87243Fe29c2453F87549e02982108a3B832952     354
0xB11c61923861695eB47CFB3CCc473173c9cF7A28     326
0x1BA8D2BBCe4C2fd4885F032E298C80aB728504B7     280
                                              ... 
0xA3991cE46a75Fb4f108b7adC758499118159069b       1
0x5eEBDcDCfb30e8b7c490ae1A3c8ff0FA25BcD151       1
0xE6ac306031aC3d6C9d3c8a2bbE34FD2a4115f0C8       1
0x2e8E5CDfF08A22CCb037cd9ecB56Ad4c96F89e78       1
0x55e1490a1878D0B61811726e2cB96560022E764c       1
Name: count, Length: 6536, dtype: int64

In [21]:
# Write to csv
combined_table.to_csv(f'rescue_order_to_owner_{snapshot_block}.csv')
combined_table.value_counts().to_csv(f'ownership_wild_acclimated_jumpport_{snapshot_block}.csv')

In [22]:
combined_table.value_counts().sum()

25434

# NFTX $MOONCAT holders

In [23]:
# https://nftx.io/vault/0x98968f0747e0a261532cacc0be296375f5c08398/info/
nftx_mooncat_addr = '0x98968f0747E0A261532cAcC0BE296375F5c08398'
nftx_mooncat_abi = '[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"moduleIndex","type":"uint256"},{"indexed":false,"internalType":"address","name":"eligibilityAddr","type":"address"}],"name":"EligibilityDeployed","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"enabled","type":"bool"}],"name":"EnableMintUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"enabled","type":"bool"}],"name":"EnableRandomRedeemUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"enabled","type":"bool"}],"name":"EnableRandomSwapUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"enabled","type":"bool"}],"name":"EnableTargetRedeemUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"enabled","type":"bool"}],"name":"EnableTargetSwapUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"manager","type":"address"}],"name":"ManagerSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"oldName","type":"string"},{"indexed":false,"internalType":"string","name":"oldSymbol","type":"string"},{"indexed":false,"internalType":"string","name":"newName","type":"string"},{"indexed":false,"internalType":"string","name":"newSymbol","type":"string"}],"name":"MetaDataChange","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256[]","name":"nftIds","type":"uint256[]"},{"indexed":false,"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"indexed":false,"internalType":"address","name":"to","type":"address"}],"name":"Minted","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256[]","name":"nftIds","type":"uint256[]"},{"indexed":false,"internalType":"uint256[]","name":"specificIds","type":"uint256[]"},{"indexed":false,"internalType":"address","name":"to","type":"address"}],"name":"Redeemed","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256[]","name":"nftIds","type":"uint256[]"},{"indexed":false,"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"indexed":false,"internalType":"uint256[]","name":"specificIds","type":"uint256[]"},{"indexed":false,"internalType":"uint256[]","name":"redeemedIds","type":"uint256[]"},{"indexed":false,"internalType":"address","name":"to","type":"address"}],"name":"Swapped","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"vaultId","type":"uint256"},{"indexed":false,"internalType":"address","name":"assetAddress","type":"address"},{"indexed":false,"internalType":"bool","name":"is1155","type":"bool"},{"indexed":false,"internalType":"bool","name":"allowAllItems","type":"bool"}],"name":"VaultInit","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"assetAddress","type":"address"},{"indexed":false,"internalType":"uint256","name":"numItems","type":"uint256"},{"indexed":false,"internalType":"address","name":"recipient","type":"address"}],"name":"VaultShutdown","type":"event"},{"inputs":[{"internalType":"string","name":"_name","type":"string"},{"internalType":"string","name":"_symbol","type":"string"},{"internalType":"address","name":"_assetAddress","type":"address"},{"internalType":"bool","name":"_is1155","type":"bool"},{"internalType":"bool","name":"_allowAllItems","type":"bool"}],"name":"__NFTXVault_init","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"allHoldings","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256[]","name":"tokenIds","type":"uint256[]"}],"name":"allValidNFTs","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"allowAllItems","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"assetAddress","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"moduleIndex","type":"uint256"},{"internalType":"bytes","name":"initData","type":"bytes"}],"name":"deployEligibilityStorage","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"disableVaultFees","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"eligibilityStorage","outputs":[{"internalType":"contract INFTXEligibility","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"enableMint","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"enableRandomRedeem","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"enableRandomSwap","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"enableTargetRedeem","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"enableTargetSwap","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"finalizeVault","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"flashFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract IERC3156FlashBorrowerUpgradeable","name":"receiver","type":"address"},{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"flashLoan","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"is1155","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"manager","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"maxFlashLoan","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256[]","name":"tokenIds","type":"uint256[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"name":"mint","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"mintFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256[]","name":"tokenIds","type":"uint256[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"internalType":"address","name":"to","type":"address"}],"name":"mintTo","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"holdingsIndex","type":"uint256"}],"name":"nftIdAt","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"operator","type":"address"},{"internalType":"address","name":"","type":"address"},{"internalType":"uint256[]","name":"","type":"uint256[]"},{"internalType":"uint256[]","name":"","type":"uint256[]"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"onERC1155BatchReceived","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"operator","type":"address"},{"internalType":"address","name":"","type":"address"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"onERC1155Received","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"operator","type":"address"},{"internalType":"address","name":"","type":"address"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"onERC721Received","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"randomRedeemFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"randomSwapFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256[]","name":"specificIds","type":"uint256[]"}],"name":"redeem","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256[]","name":"specificIds","type":"uint256[]"},{"internalType":"address","name":"to","type":"address"}],"name":"redeemTo","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"}],"name":"retrieveTokens","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_mintFee","type":"uint256"},{"internalType":"uint256","name":"_randomRedeemFee","type":"uint256"},{"internalType":"uint256","name":"_targetRedeemFee","type":"uint256"},{"internalType":"uint256","name":"_randomSwapFee","type":"uint256"},{"internalType":"uint256","name":"_targetSwapFee","type":"uint256"}],"name":"setFees","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_manager","type":"address"}],"name":"setManager","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bool","name":"_enableMint","type":"bool"},{"internalType":"bool","name":"_enableRandomRedeem","type":"bool"},{"internalType":"bool","name":"_enableTargetRedeem","type":"bool"},{"internalType":"bool","name":"_enableRandomSwap","type":"bool"},{"internalType":"bool","name":"_enableTargetSwap","type":"bool"}],"name":"setVaultFeatures","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"string","name":"name_","type":"string"},{"internalType":"string","name":"symbol_","type":"string"}],"name":"setVaultMetadata","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"}],"name":"shutdown","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes4","name":"interfaceId","type":"bytes4"}],"name":"supportsInterface","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256[]","name":"tokenIds","type":"uint256[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"internalType":"uint256[]","name":"specificIds","type":"uint256[]"}],"name":"swap","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256[]","name":"tokenIds","type":"uint256[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"internalType":"uint256[]","name":"specificIds","type":"uint256[]"},{"internalType":"address","name":"to","type":"address"}],"name":"swapTo","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"targetRedeemFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"targetSwapFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalHoldings","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"sender","type":"address"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"vaultFactory","outputs":[{"internalType":"contract INFTXVaultFactory","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"vaultFees","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"vaultId","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"pure","type":"function"}]'
nftx_mooncat_deploy_block = 12740084

In [24]:
nftx_mooncat_contract = w3.eth.contract(address=nftx_mooncat_addr, abi=nftx_mooncat_abi)
nftx_mooncat_transfer_events = nftx_mooncat_contract.events.Transfer.create_filter(fromBlock=nftx_mooncat_deploy_block, toBlock=snapshot_block).get_all_entries()

In [26]:
len(nftx_mooncat_transfer_events)

4865

In [27]:
nftx_mooncat_holders = {}

for event in nftx_mooncat_transfer_events:
    from_addr = event['args']['from']
    to_addr = event['args']['to']
    value = event['args']['value']

    # Decrease balance of sender
    if from_addr != "0x0000000000000000000000000000000000000000":  # Check for token creation event
        nftx_mooncat_holders[from_addr] = nftx_mooncat_holders.get(from_addr, 0) - value

    # Increase balance of receiver
    nftx_mooncat_holders[to_addr] = nftx_mooncat_holders.get(to_addr, 0) + value

nftx_mooncat_holders = {address: balance for address, balance in nftx_mooncat_holders.items() if balance > 0}

In [28]:
print(f'Unique wallets holding $MOONCAT at snapshot block {snapshot_block}: {len(nftx_mooncat_holders)}')

Unique wallets holding $MOONCAT at snapshot block 18780000: 91


In [29]:
display(nftx_mooncat_holders)

{'0x4586554a30148B8F4F3AB17E57C430eE193698Ec': 222580593580876119,
 '0x0aa1E808bbA7CDE5210D13D23eE726de37b8a8bb': 40260285943038239093,
 '0x0000000000000000000000000000000000000000': 772405743000000000000,
 '0xB23453d9Fa7ba3578F9218F7e10864BF8D1ce3CA': 1618910967319362,
 '0x7AE9D7Ee8489cAD7aFc84111b8b185EE594Ae090': 1000,
 '0xaB502a2b41e5939450C88B0F183114c1cAE2f448': 4546598017774509,
 '0x869959AfEF3B6B801D31635c62C27521689b8652': 1,
 '0x07e0899e179b7C3e5f601953Ee97565E6F2739aB': 78057659184068948,
 '0xF133B16adD084CB99a4915D589A2C43C475ba068': 233134205029334292,
 '0xA60e156cfE32BaE390ce1749d5b6cdf47c3f966F': 50000000000000000,
 '0x1824655211eF781631C89372fE56f030Cbb137DC': 1133347341545393,
 '0x293aa3eB1C0d93574bA3Fc8c0857002775C4B905': 2,
 '0x1d6E8BAC6EA3730825bde4B005ed7B2B39A2932d': 1,
 '0x8773058780415672ed7eD9259D39777637c28425': 6466409950788361,
 '0x11eDedebF63bef0ea2d2D071bdF88F71543ec6fB': 8681750000000000,
 '0x2a811dA74F22B3222F67cF034467536b97494f9c': 19217315188419980826

# NFTX $MCAT17 holders

In [30]:
# https://nftx.io/vault/0xa8b42c82a628dc43c2c2285205313e5106ea2853/info/
nftx_mcat17_addr = '0xA8b42C82a628DC43c2c2285205313e5106EA2853'
nftx_mcat17_abi = '[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"moduleIndex","type":"uint256"},{"indexed":false,"internalType":"address","name":"eligibilityAddr","type":"address"}],"name":"EligibilityDeployed","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"enabled","type":"bool"}],"name":"EnableMintUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"enabled","type":"bool"}],"name":"EnableRandomRedeemUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"enabled","type":"bool"}],"name":"EnableRandomSwapUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"enabled","type":"bool"}],"name":"EnableTargetRedeemUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"enabled","type":"bool"}],"name":"EnableTargetSwapUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"manager","type":"address"}],"name":"ManagerSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"oldName","type":"string"},{"indexed":false,"internalType":"string","name":"oldSymbol","type":"string"},{"indexed":false,"internalType":"string","name":"newName","type":"string"},{"indexed":false,"internalType":"string","name":"newSymbol","type":"string"}],"name":"MetaDataChange","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256[]","name":"nftIds","type":"uint256[]"},{"indexed":false,"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"indexed":false,"internalType":"address","name":"to","type":"address"}],"name":"Minted","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256[]","name":"nftIds","type":"uint256[]"},{"indexed":false,"internalType":"uint256[]","name":"specificIds","type":"uint256[]"},{"indexed":false,"internalType":"address","name":"to","type":"address"}],"name":"Redeemed","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256[]","name":"nftIds","type":"uint256[]"},{"indexed":false,"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"indexed":false,"internalType":"uint256[]","name":"specificIds","type":"uint256[]"},{"indexed":false,"internalType":"uint256[]","name":"redeemedIds","type":"uint256[]"},{"indexed":false,"internalType":"address","name":"to","type":"address"}],"name":"Swapped","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"vaultId","type":"uint256"},{"indexed":false,"internalType":"address","name":"assetAddress","type":"address"},{"indexed":false,"internalType":"bool","name":"is1155","type":"bool"},{"indexed":false,"internalType":"bool","name":"allowAllItems","type":"bool"}],"name":"VaultInit","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"assetAddress","type":"address"},{"indexed":false,"internalType":"uint256","name":"numItems","type":"uint256"},{"indexed":false,"internalType":"address","name":"recipient","type":"address"}],"name":"VaultShutdown","type":"event"},{"inputs":[{"internalType":"string","name":"_name","type":"string"},{"internalType":"string","name":"_symbol","type":"string"},{"internalType":"address","name":"_assetAddress","type":"address"},{"internalType":"bool","name":"_is1155","type":"bool"},{"internalType":"bool","name":"_allowAllItems","type":"bool"}],"name":"__NFTXVault_init","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"allHoldings","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256[]","name":"tokenIds","type":"uint256[]"}],"name":"allValidNFTs","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"allowAllItems","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"assetAddress","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"moduleIndex","type":"uint256"},{"internalType":"bytes","name":"initData","type":"bytes"}],"name":"deployEligibilityStorage","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"disableVaultFees","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"eligibilityStorage","outputs":[{"internalType":"contract INFTXEligibility","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"enableMint","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"enableRandomRedeem","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"enableRandomSwap","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"enableTargetRedeem","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"enableTargetSwap","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"finalizeVault","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"flashFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract IERC3156FlashBorrowerUpgradeable","name":"receiver","type":"address"},{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"flashLoan","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"is1155","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"manager","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"maxFlashLoan","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256[]","name":"tokenIds","type":"uint256[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"name":"mint","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"mintFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256[]","name":"tokenIds","type":"uint256[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"internalType":"address","name":"to","type":"address"}],"name":"mintTo","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"holdingsIndex","type":"uint256"}],"name":"nftIdAt","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"operator","type":"address"},{"internalType":"address","name":"","type":"address"},{"internalType":"uint256[]","name":"","type":"uint256[]"},{"internalType":"uint256[]","name":"","type":"uint256[]"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"onERC1155BatchReceived","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"operator","type":"address"},{"internalType":"address","name":"","type":"address"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"onERC1155Received","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"operator","type":"address"},{"internalType":"address","name":"","type":"address"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"onERC721Received","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"randomRedeemFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"randomSwapFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256[]","name":"specificIds","type":"uint256[]"}],"name":"redeem","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256[]","name":"specificIds","type":"uint256[]"},{"internalType":"address","name":"to","type":"address"}],"name":"redeemTo","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"}],"name":"retrieveTokens","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_mintFee","type":"uint256"},{"internalType":"uint256","name":"_randomRedeemFee","type":"uint256"},{"internalType":"uint256","name":"_targetRedeemFee","type":"uint256"},{"internalType":"uint256","name":"_randomSwapFee","type":"uint256"},{"internalType":"uint256","name":"_targetSwapFee","type":"uint256"}],"name":"setFees","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_manager","type":"address"}],"name":"setManager","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bool","name":"_enableMint","type":"bool"},{"internalType":"bool","name":"_enableRandomRedeem","type":"bool"},{"internalType":"bool","name":"_enableTargetRedeem","type":"bool"},{"internalType":"bool","name":"_enableRandomSwap","type":"bool"},{"internalType":"bool","name":"_enableTargetSwap","type":"bool"}],"name":"setVaultFeatures","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"string","name":"name_","type":"string"},{"internalType":"string","name":"symbol_","type":"string"}],"name":"setVaultMetadata","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"}],"name":"shutdown","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes4","name":"interfaceId","type":"bytes4"}],"name":"supportsInterface","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256[]","name":"tokenIds","type":"uint256[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"internalType":"uint256[]","name":"specificIds","type":"uint256[]"}],"name":"swap","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256[]","name":"tokenIds","type":"uint256[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"},{"internalType":"uint256[]","name":"specificIds","type":"uint256[]"},{"internalType":"address","name":"to","type":"address"}],"name":"swapTo","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"targetRedeemFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"targetSwapFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalHoldings","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"sender","type":"address"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"vaultFactory","outputs":[{"internalType":"contract INFTXVaultFactory","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"vaultFees","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"vaultId","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"pure","type":"function"}]'
nftx_mcat17_deploy_block = 14626075

In [31]:
nftx_mcat17_contract = w3.eth.contract(address=nftx_mcat17_addr, abi=nftx_mcat17_abi)
nftx_mcat17_transfer_events = nftx_mcat17_contract.events.Transfer.create_filter(fromBlock=nftx_mcat17_deploy_block, toBlock=snapshot_block).get_all_entries()
print(len(nftx_mcat17_transfer_events))

761


In [32]:
nftx_mcat17_holders = {}

for event in nftx_mcat17_transfer_events:
    from_addr = event['args']['from']
    to_addr = event['args']['to']
    value = event['args']['value']

    # Decrease balance of sender
    if from_addr != "0x0000000000000000000000000000000000000000":  # Check for token creation event
        nftx_mcat17_holders[from_addr] = nftx_mcat17_holders.get(from_addr, 0) - value

    # Increase balance of receiver
    nftx_mcat17_holders[to_addr] = nftx_mcat17_holders.get(to_addr, 0) + value

nftx_mcat17_holders = {address: balance for address, balance in nftx_mcat17_holders.items() if balance > 0}

In [33]:
print(f'Unique wallets holding $MCAT17 at snapshot block {snapshot_block}: {len(nftx_mcat17_holders)}')

Unique wallets holding $MCAT17 at snapshot block 18780000: 32


In [34]:
nftx_mcat17_holders

{'0x11a4975b0eB5a37ec637fe395AD767ca4D4AEF7f': 16294618550501151379,
 '0x29f033720d84d0F85c1becCf80E51449C06Ca2d2': 15785581931017758088,
 '0xB123Ab41aa158f78694329D1788B885D009e18AD': 5385964080367491,
 '0x4f9295A18334a3dB3cc8C158bCb186706d047F31': 1327965236116846177,
 '0x0000000000000000000000000000000000000000': 60000000000000000000,
 '0xC5C5d19FccC22B2D0aB1938cE43975C31839e34a': 730068785317404,
 '0xE8c060F8052E07423f71D445277c61AC5138A2e5': 1,
 '0xdcccbC712de62D3438A6620Dba01320E283eC006': 3688154292579876,
 '0x007933790a4f00000099e9001629d9fE7775B800': 70000000000000,
 '0xB11c61923861695eB47CFB3CCc473173c9cF7A28': 535138409128084656,
 '0xbbc53022Af15Bb973AD906577c84784c47C14371': 19905929459335814,
 '0x00000000003b3cc22aF3aE1EAc0440BcEe416B40': 1972101182397,
 '0x00000000500e2fece27a7600435d0C48d64E0C00': 8589934592,
 '0xd10C70000bb40B1c4579b522a7597eB8db2d10A7': 10000000000000,
 '0x1189f4016460543Ac15Cfb71b393c08682073a76': 2001140211262840202,
 '0x20C467db9B9Fe0fA39D879b3f23c4

# combine everything

In [35]:
df = combined_table.value_counts().reset_index().copy()
df.columns = ['address','num_cats']
df.shape[0]

6536

In [36]:
# add on nftx_mcat17_holders and nftx_mooncat_holders
df['$mooncat_balance'] = df['address'].map(nftx_mooncat_holders).fillna(0)

missing_mcat_addresses = [address for address in nftx_mooncat_holders if address not in df['address'].values]
new_mcat_rows = pd.DataFrame([{'address': address, '$mooncat_balance':nftx_mooncat_holders[address]} for address in missing_mcat_addresses])
df = pd.concat([df, new_mcat_rows], ignore_index=True)

df['$mcat17_balance'] = df['address'].map(nftx_mcat17_holders).fillna(0)
missing_mcat17_addresses = [address for address in nftx_mcat17_holders if address not in df['address'].values]
new_mcat17_rows = pd.DataFrame([{'address': address, '$mcat17_balance':nftx_mcat17_holders[address]} for address in missing_mcat17_addresses])
df = pd.concat([df, new_mcat17_rows], ignore_index=True)

df = df.drop_duplicates()

df['$mooncat_balance'] = df['$mooncat_balance'] / 1e18
df['$mcat17_balance'] = df['$mcat17_balance'] / 1e18
df['num_cats'] = df['num_cats'].fillna(0)
df

,address,num_cats,$mooncat_balance,$mcat17_balance
0,0x7C40c393DC0f283F318791d746d894DdD3693572,1355.0,0.0,0.0
1,0xe307C2d3236bE4706E5D7601eE39F16d796d8195,1098.0,0.0,0.0
2,0xEF87243Fe29c2453F87549e02982108a3B832952,354.0,0.0,0.0
3,0xB11c61923861695eB47CFB3CCc473173c9cF7A28,326.0,0.0,0.535138
4,0x1BA8D2BBCe4C2fd4885F032E298C80aB728504B7,280.0,0.008877,0.0
...,...,...,...,...
6611,0x6b75d8AF000000e20B7a7DDf000Ba900b4009A80,0.0,NaN,0.0
6612,0x35023c4828B86F1eF3cD6405E31dB8301a0Ea133,0.0,NaN,0.000001
6613,0xf34FB129f2d2767D8202E349439967C2D1F98CE4,0.0,NaN,0.000001
6614,0xDc790939164E1aD7962B1568Ecb727d8463608DB,0.0,NaN,0.002895


# check which addresses are contracts

In [37]:
df['is_contract'] = False
df

,address,num_cats,$mooncat_balance,$mcat17_balance,is_contract
0,0x7C40c393DC0f283F318791d746d894DdD3693572,1355.0,0.0,0.0,False
1,0xe307C2d3236bE4706E5D7601eE39F16d796d8195,1098.0,0.0,0.0,False
2,0xEF87243Fe29c2453F87549e02982108a3B832952,354.0,0.0,0.0,False
3,0xB11c61923861695eB47CFB3CCc473173c9cF7A28,326.0,0.0,0.535138,False
4,0x1BA8D2BBCe4C2fd4885F032E298C80aB728504B7,280.0,0.008877,0.0,False
...,...,...,...,...,...
6611,0x6b75d8AF000000e20B7a7DDf000Ba900b4009A80,0.0,NaN,0.0,False
6612,0x35023c4828B86F1eF3cD6405E31dB8301a0Ea133,0.0,NaN,0.000001,False
6613,0xf34FB129f2d2767D8202E349439967C2D1F98CE4,0.0,NaN,0.000001,False
6614,0xDc790939164E1aD7962B1568Ecb727d8463608DB,0.0,NaN,0.002895,False


In [38]:
for idx,row in tqdm(df.iterrows(),total=df.shape[0]):
    if w3.eth.get_code(row['address']).hex()!='0x':
        df.loc[idx,'is_contract'] = True
    time.sleep(0.01)

  0%|          | 0/6616 [00:00<?, ?it/s]

In [39]:
# These are contracts...
pd.set_option('display.max_rows',150)
print(df[df['is_contract']].shape[0])
display(df[df['is_contract']])

122


,address,num_cats,$mooncat_balance,$mcat17_balance,is_contract
0,0x7C40c393DC0f283F318791d746d894DdD3693572,1355.0,0.0,0.0,True
6,0x98968f0747E0A261532cAcC0BE296375F5c08398,243.0,0.0,0.0,True
10,0x035eA90E53bbDa2927b798b094bAB38EA21010Be,169.0,0.0,0.0,True
27,0x7AAA3C62436F29b047821106e1D384D373C543a4,81.0,0.0,0.0,True
78,0xA8b42C82a628DC43c2c2285205313e5106EA2853,36.0,0.0,0.0,True
184,0x67BDcD02705CEcf08Cb296394DB7d6Ed00A496F9,17.0,0.0,0.0,True
271,0xcDA0Af6dC4fEDDd9D7691E91DA61d02Dc1C4e773,13.0,0.0,0.0,True
317,0x2221718322473E9516238D6121506a464E85dA49,11.0,0.0,0.0,True
444,0xc6400A5584db71e41B0E5dFbdC769b54B91256CD,8.0,0.0,0.0,True
480,0x25C13c0F5c957a7c50966De8141b4Cd86627dD39,8.0,0.0,0.0,True


In [40]:
display(df[df['address']=='0x0000000000000000000000000000000000000000'])
df = df.drop(df[df['address']=='0x0000000000000000000000000000000000000000'].index)
df

,address,num_cats,$mooncat_balance,$mcat17_balance,is_contract
6538,0x0000000000000000000000000000000000000000,0.0,772.405743,60.0,False


,address,num_cats,$mooncat_balance,$mcat17_balance,is_contract
0,0x7C40c393DC0f283F318791d746d894DdD3693572,1355.0,0.0,0.0,True
1,0xe307C2d3236bE4706E5D7601eE39F16d796d8195,1098.0,0.0,0.0,False
2,0xEF87243Fe29c2453F87549e02982108a3B832952,354.0,0.0,0.0,False
3,0xB11c61923861695eB47CFB3CCc473173c9cF7A28,326.0,0.0,0.535138,False
4,0x1BA8D2BBCe4C2fd4885F032E298C80aB728504B7,280.0,0.008877,0.0,False
...,...,...,...,...,...
6611,0x6b75d8AF000000e20B7a7DDf000Ba900b4009A80,0.0,NaN,0.0,True
6612,0x35023c4828B86F1eF3cD6405E31dB8301a0Ea133,0.0,NaN,0.000001,False
6613,0xf34FB129f2d2767D8202E349439967C2D1F98CE4,0.0,NaN,0.000001,False
6614,0xDc790939164E1aD7962B1568Ecb727d8463608DB,0.0,NaN,0.002895,False


In [41]:
df.to_csv(f'snapshot_{snapshot_block}.csv',index=True)